In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib
import time
import datetime
from IPython import display
import random
import cv2


""" processed_dir = ["0001 - bleach", "0002 - one piece","0003 - marvel","0004 - DC", "0005 - Love", "0006 - soma", 
                        "0007 - ishoujo", "0008 - vagabond", "0009 - vampire r", "0010 - gintama", "0011 - gantz",
                         "0012 - chainsaw", "0013 - death note", "0014 - batman" ] """
""" processed_dir = [""] """

processed_dir = ["0001 - bleach", "0002 - one piece","0003 - marvel","0004 - DC", "0005 - Love", "0006 - soma", 
                        "0007 - ishoujo", "0008 - vagabond","0010 - gintama", "0011 - gantz", "0012 - chainsaw",
                         "0013 - death note", "0014 - batman" ]


data_directory = 'D:\\Deletar\\pictures'
test_directory = 'G:\\Deletar\\pictures_conv'
tf.random.set_seed(42) 
SIZE = [512,512]
print(tf. __version__)

2.6.0


In [3]:
def resize_image(data):
    resized_img = tf.image.resize(
    images=data,
    size=SIZE,
    method=tf.image.ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=True,    
    name=None
    )    
    return resized_img

def save_f(img,filename,contI):
    dir_save = ""
    if (contI % 5 != 0):
        dir_save = f"{test_directory}\\train\\{filename}.jpg"
    else:
        dir_save = f"{test_directory}\\test\\{filename}.jpg"
    tf.keras.preprocessing.image.save_img(dir_save, img)
    
#if (os.path.exists(os.path.join(f"{data_directory}\\{folder}",filename+".jpg"))):    

def load_images_from_folder(folder):
    images = []
    cont = 0
    cont2 = 0    
    for folder in os.listdir(folder):
        if not folder in processed_dir:
            for filename in os.listdir(os.path.join(f"{data_directory}\\{folder}")):        
                cont2 = cont2 + 1
                if (not os.path.exists(os.path.join(f"{data_directory}\\{folder}\\",filename+".jpg"))):                        
                    try:
                        img1 = resize_image(plt.imread(os.path.join(f"{data_directory}\\{folder}\\",filename)))
                        img2 =  tf.image.rgb_to_grayscale(img1)
                        img2 = np.concatenate((img2,img2,img2),axis=-1)
                        img_merged = tf.concat([img1, img2], 1)
                        if img_merged is not None:
                            save_f(img_merged,filename,cont2)
                    except:
                            cont = cont + 1
                            print(f"Error in {cont}")
                print(cont2)
    return images

""" X_test = load_images_from_folder(data_directory) # join two images """


def save_bw_cl(img,filename,contI, img2):
    dir_save = ""
    if (contI % 10 != 0):
        dir_save = f"{test_directory}\\train_new\\color\\{filename}.jpg"
        dir_save2 = f"{test_directory}\\train_new\\bw\\{filename}.jpg"
    else:
        dir_save = f"{test_directory}\\test_new\\color\\{filename}.jpg"
        dir_save2 = f"{test_directory}\\test_new\\bw\\{filename}.jpg"
    tf.keras.preprocessing.image.save_img(dir_save, img)
    tf.keras.preprocessing.image.save_img(dir_save2, img2)

def get_bw_from_folder(folder):
    images = []
    cont = 0
    cont2 = 0    
    for folder in os.listdir(folder):
        if not folder in processed_dir:
            for filename in os.listdir(os.path.join(f"{data_directory}\\{folder}")):        
                cont2 = cont2 + 1
                
                """ if (not os.path.exists(os.path.join(f"{data_directory}\\{folder}\\",filename))): """
                               
                try:
                    img = plt.imread(os.path.join(f"{data_directory}\\{folder}\\",filename))                    
                    img = resize_image(img)     
                    img2 = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY) # Input IMG is RGB
                    img2 = cv2.merge([img2,img2,img2])    
                        
                    if img is not None:
                        save_bw_cl(img,filename,cont2, img2)
                            
                except Exception as e:
                    cont = cont + 1                            
                    print(e)
                
    return images

X_test = get_bw_from_folder(data_directory)
